In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


## Load  Data

In [2]:
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
df.drop(columns=['Id'], inplace=True)

display(df.head())

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


### Train Test Split

In [3]:
from sklearn.model_selection import train_test_split

target = df['Class']
df = df.drop(columns=['Class'])

train_x, val_x, train_y, val_y = train_test_split(df, target, test_size=0.2, random_state=42)


### Impute Missing/NAN Data

In [5]:
from sklearn.impute import SimpleImputer

feature_list = train_x.columns.tolist()

# Handle numeric features
numeric_features = feature_list.copy()
numeric_features.remove('EJ')  # EJ is the only categorical feature
num_imputer = SimpleImputer(strategy='mean')
num_imputer.fit(train_x[numeric_features])
train_x[numeric_features] = num_imputer.transform(train_x[numeric_features])
val_x[numeric_features] = num_imputer.transform(val_x[numeric_features])

# Handle categorical features
categorical_features = ['EJ']
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer.fit(train_x[categorical_features])
train_x[categorical_features] = cat_imputer.transform(train_x[categorical_features])
val_x[categorical_features] = cat_imputer.transform(val_x[categorical_features])

# Encode 'EJ' as binary 0/1 (A=0, B=1)
train_x['EJ'] = (train_x['EJ'] == 'B').astype(int)
val_x['EJ'] = (val_x['EJ'] == 'B').astype(int)

### Scale the numeric Data

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x[numeric_features])
train_x[numeric_features] = scaler.transform(train_x[numeric_features])

display(train_x.head())

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
530,3.467721,0.898806,-0.252523,7.404421,3.479259,0.626453,-0.318591,0.277248,-0.102247,-0.111570,...,-0.140707,1.487529,-0.063470,-0.122322,0.137313,-0.426529,-0.219871,-0.474489,3.013722,-0.818696
363,-0.158606,0.158262,0.111673,-0.411824,-0.173278,-0.076455,-0.318591,0.043742,-0.067391,-0.696674,...,-0.247273,-0.123113,-0.053902,-0.033191,-0.114374,-0.153680,-0.749480,0.001749,-0.563045,-0.810696
177,-0.054000,-0.467750,-0.199919,0.031757,-0.173278,0.629849,-0.318591,-0.980533,-0.102247,-1.113511,...,-1.159005,-0.228830,-0.063470,-0.258365,-0.913572,-0.426529,-0.712492,2.165110,-1.065435,-0.812208
312,-0.585745,-0.528526,-0.252523,-0.289348,0.299719,-0.633348,1.766762,-0.204089,-0.102247,-0.240922,...,0.988104,-0.439374,-0.063470,-0.253674,-0.262756,-0.426529,-0.491888,0.787312,-0.124169,1.320516
199,-0.629331,-1.019059,-0.212834,-0.424194,-0.173278,-0.378671,-0.318591,-0.443326,-0.102247,-0.619721,...,-0.261087,-0.037448,-0.054108,-0.117631,1.466179,-0.426529,-0.382173,-1.103859,-0.501847,-0.793778


### SVC MODEL

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

model = SVC(kernel='rbf', C=1.0, gamma='scale')
model.fit(train_x, train_y)
pred = model.predict(val_x)
accuracy = accuracy_score(val_y, pred)
print("Accuracy:", accuracy)


Accuracy: 0.8145161290322581


### Export the Predictions for SVM Model

In [14]:
# test_df = pd.read_csv('data/test.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

# Prepare test features
test_x = test_df.drop(columns=['Id']).copy()

# Impute numeric features
test_x[numeric_features] = num_imputer.transform(test_x[numeric_features])

# Impute categorical 'EJ' and encode to 0/1
test_x[categorical_features] = cat_imputer.transform(test_x[categorical_features])
test_x['EJ'] = (test_x['EJ'] == 'B').astype(int)

# Scale numeric features
test_x[numeric_features] = scaler.transform(test_x[numeric_features])

# Predict probabilities (binary)
prediction = model.decision_function(test_x)


submission = pd.DataFrame({
    'Id': test_df['Id'].values,
    'class_0': 1- prediction,
    'class_1': prediction
})

submission.to_csv('submission.csv', index=False)